In [ ]:
!pip install -q tensorflow scikit-learn pandas matplotlib ipywidgets

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv('Mentalillness.csv')#Φόρτωση δεδομένων
df_clean = df.drop(columns=["ID", "Intrusive memories or flashback", "Sleep disturbance.1"], errors="ignore")
#Στήλες στόχοι
target_columns = ['Bipolar disorder', 'Schizophrenia', 'Depression', 'Anxiety disorder', 'PTSD']
X = df_clean.drop(columns=target_columns)
y = df_clean[target_columns]

#Κανονικοποίηση
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
#Μοντέλο
model = Sequential([
    Dense(128, input_dim=X.shape[1], activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(target_columns), activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#Εκπαίδευση μοντέλου
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)
#Αναφορά απόδοσης
preds = model.predict(X_test)
preds_binary = (preds > 0.5).astype(int)
print("\n Report By Deviation:")
print(classification_report(y_test, preds_binary, target_names=target_columns))

#Τέστ στον χρήστη
questions = list(X.columns)
answers = []
index = 0
question_label = widgets.Label()
radio_buttons = widgets.RadioButtons(options=['Yes', 'No'], description='Answer:', value=None)
result_output = widgets.HTML()

intro_label = widgets.HTML("<h3>🧠 DSM-5 Mental Health Test</h3> <p> Answer the questions below:</p> 🧠")

def on_radio_change(change):
    global index, answers
    if change['new'] is not None:
        answers.append(1 if change['new'] == 'Yes' else 0)
        if index + 1 < len(questions):
            index += 1
            question_label.value = questions[index]
            radio_buttons.value = None
        else:
            show_results()

def show_results():
    global answers
    if len(answers) < X.shape[1]:
        answers.extend([0] * (X.shape[1] - len(answers)))
    user_input_scaled = scaler.transform([answers])
    pred = model.predict(user_input_scaled)[0]
    results = [
        f"<b>{target_columns[i]}:</b> {'Yes ✅' if pred[i] > 0.5 else 'No ❌'} (Probability: {pred[i]:.2f})"
        for i in range(len(target_columns))]
    result_output.value = '<br>'.join(results)
    question_label.value = "Results:"
    radio_buttons.layout.display = 'none'

radio_buttons.observe(on_radio_change, names='value')

# Εμφάνιση αρχικής εισαγωγής και πρώτης ερώτησης
question_label.value = questions[index]
display(intro_label, question_label, radio_buttons, result_output)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

 Report By Deviation:
                  precision    recall  f1-score   support

Bipolar disorder       1.00      1.00      1.00       586
   Schizophrenia       1.00      1.00      1.00       640
      Depression       1.00      1.00      1.00       361
Anxiety disorder       1.00      1.00      1.00       338
            PTSD       0.82      0.59      0.69        39

       micro avg       1.00      0.99      0.99      1964
       macro avg       0.96      0.92      0.94      1964
    weighted avg       0.99      0.99      0.99      1964
     samples avg       0.99      0.99      0.99      1964



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


HTML(value='<h3>🧠 DSM-5 Mental Health Test</h3> <p> Answer the questions below:</p> ')

Label(value='Inflated self-esteem')

RadioButtons(description='Answer:', options=('Yes', 'No'), value=None)

HTML(value='')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
